In [ ]:
import torch

import matplotlib.pyplot as plt
import torchmetrics as tm

from json import load, dump

from config import *
from stages import *
from train import *
from data.ssa import SSA

from data.util import crop_q_between, split_weekdays_and_weekends

%reload_ext autoreload
%autoreload 2

In [ ]:
mse = tm.MeanSquaredError().to(CONFIG.device)
mape = tm.MeanAbsolutePercentageError().to(CONFIG.device)
mat_q = CONFIG.load('mat_q.pt')
CONFIG.alpha = 0.2
CONFIG.alpha = 1000
mat_c_all, mat_x_repr, nonempty, representatives = compress_data(
    mat_q.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)
mat_q = mat_q[:, mat_q.sum(dim=0) > 0]
normalizing_constants_all = mat_q.max(dim=0)[0]
mat_q /= normalizing_constants_all
mat_q

In [ ]:
CONFIG.save(mat_q, 'mat_q_nonzero.pt')
mat_q.shape

##### Compare MSE and MAPE losses for different parameter configurations

Pre-train DBN and Train DBN attaching KELM on each epoch

In [ ]:
results_wd = {}
results_we = {}

ssa = SSA(4, [2, 2], CONFIG.device)

CONFIG.spectral_threshold = 1440
CONFIG.dbn_hidden_layer_sizes = [80, 80, 80]

gamma = 1
reg_coeff = 1


for P in tqdm(range(110, 210, 10)):#[40, 126.5, 400, 1265, 4000, 12650, 40000]):
  results_P = {}
  CONFIG.spectral_threshold = P
  for N in range(4, 324, 80):
    CONFIG.dbn_hidden_layer_sizes = [N, N, N]

    mat_c_all, mat_x_all, nonempty, representatives = compress_data(
        mat_q.abs(), CONFIG.read_period, CONFIG.train_period, CONFIG.alpha)
    mat_q_trend_all, mat_q_resid_all = preprocess_data(
        CONFIG.spectral_threshold, mat_c_all)
    mat_q_resid_all = crop_q_between(mat_q_resid_all, CONFIG.read_period,
                                      CONFIG.train_period).real
    mat_c_all = crop_q_between(
        mat_c_all, CONFIG.read_period, CONFIG.train_period)
    
    mat_q_trend_all

    for section_index, section_number in tqdm(enumerate(representatives.tolist())):
      mat_q_trend = mat_q_trend_all[:, section_index][:, None].real
      mat_q_resid = mat_q_resid_all[:, section_index][:, None].real

      norm_const = normalizing_constants_all[section_number].item()

      (_, val_trend_wd_dataloader, _), (_, val_trend_we_dataloader, _) = \
        crop_and_split_mat(mat_q_trend, CONFIG, separate_weekends=False)
      mat_c_wd_datasets, mat_c_we_datasets = split_mat(mat_q_resid, CONFIG)

      del mat_q_resid

      dbn, kelm, val_resid_wd_dataloader = train_with_config(
        CONFIG, mat_c_wd_datasets, dbn_training_epochs=100, stride=1,
        gamma=gamma, reg_coeff=reg_coeff
      )
      prefix = f'alpha{CONFIG.alpha}_p{P}_n{N}_sec{section_number}_'\
        .replace('.', '_')
      CONFIG.save(dbn.state_dict(), prefix + 'dbn.pt')
      CONFIG.save(kelm.state_dict(), prefix + 'kelm.pt')

      mse_loss_trend = torch.tensor([0.,]).to(CONFIG.device)
      mse_loss_resid = torch.tensor([0.,]).to(CONFIG.device)
      mse_loss_overall = torch.tensor([0.,]).to(CONFIG.device)
      mse_loss_trend_unnorm = torch.tensor([0.,]).to(CONFIG.device)
      mse_loss_resid_unnorm = torch.tensor([0.,]).to(CONFIG.device)
      mse_loss_overall_unnorm = torch.tensor([0.,]).to(CONFIG.device)

      trend_y = torch.tensor([0.,]).to(CONFIG.device)
      resid_y = torch.tensor([0.,]).to(CONFIG.device)
      overall_y = torch.tensor([0.,]).to(CONFIG.device)

      iter_trend = iter(val_trend_wd_dataloader)
      n_samples = 0

      resid = {'y': [], 'pred': []}
      trend = {'y': [], 'pred': []}
      overall = {'y': [], 'pred': []}
      for X_resid, y_resid in val_resid_wd_dataloader:
          resid_y += y_resid.item()
          X_trend, y_trend = next(iter_trend)
          trend_y += y_trend.item()
          pred_trend = ssa.forecast(
              X_trend.squeeze(0).T, 1).sum(0)[-1][None]
          mse_loss_trend += mse(pred_trend, y_trend)
          mse_loss_trend_unnorm += mse(pred_trend * norm_const, y_trend * norm_const)
          trend['y'].append(y_trend.item() * norm_const)
          trend['pred'].append(pred_trend.item() * norm_const)

          pred_resid = dbn(X_resid).squeeze(0)
          pred_resid = kelm(pred_resid).T
          mse_loss_resid += mse(pred_resid, y_resid)
          mse_loss_resid_unnorm += mse(pred_resid * norm_const, y_resid * norm_const)
          resid['y'].append(y_resid.item() * norm_const)
          resid['pred'].append(pred_resid.item() * norm_const)

          pred = pred_trend + pred_resid

          y = y_trend + y_resid
          overall_y += y.item()
          overall['y'].append(y.item() * norm_const)
          overall['pred'].append(pred.item() * norm_const)

          mse_loss_overall += mse(pred, y).item()
          mse_loss_overall_unnorm += mse(pred * norm_const, y * norm_const).item()
          n_samples += 1

      mse_loss_trend /= n_samples
      mse_loss_resid /= n_samples
      mse_loss_overall /= n_samples
      mse_loss_resid_unnorm /= n_samples
      mse_loss_trend_unnorm /= n_samples
      mse_loss_overall_unnorm /= n_samples

      resid_y /= n_samples
      trend_y /= n_samples
      overall_y /= n_samples

      results_P[section_number] = {
          'resid': resid,
          'trend': trend,
          'overall': overall
      }

      with open(f'results_wd_{section_number}.json', 'w') as file:
        dump(results_P[section_number], file)

      del dbn
      del kelm

  results_wd[P] = results_P

In [ ]:
import json
with open('results_wd_norm_adjust.json', 'w') as file:
  json.dump(results_we, file)